<a href="https://colab.research.google.com/github/brady-at-claradata/ElegantRL/blob/master/PPO-Stock-Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






In [1]:
import torch

In [2]:
torch.cuda.get_device_properties(0)


_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)

# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

In [3]:
!pip install git+https://github.com/brady-at-claradata/ElegantRL.git
!pip install yfinance stockstats

  Cloning https://github.com/brady-at-claradata/ElegantRL.git to /tmp/pip-req-build-pix0hvu0
  Running command git clone -q https://github.com/brady-at-claradata/ElegantRL.git /tmp/pip-req-build-pix0hvu0
     |████████████████████████████████| 79.0 MB 58 kB/s 
     |████████████████████████████████| 448 kB 59.9 MB/s 
  Created wheel for elegantrl: filename=elegantrl-0.3.1-py3-none-any.whl size=65629 sha256=f1b456d6a4781e30430b7ce5336652a947f018d8c5be6cb8d9cccb74abccf381
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ymp3dq3/wheels/16/7f/86/3423008aa0ad8c1bc651630fe0c45ed3efe0ec6db54a5fcf8f



  Created wheel for pybullet: filename=pybullet-3.1.7-cp37-cp37m-linux_x86_64.whl size=89750883 sha256=021f6137c4e6171520c5b8d11b95a9f6b7071003165ed0e40a9a45c9cf1b68ed
  Stored in directory: /root/.cache/pip/wheels/70/1c/62/86c8b68885c24123d87c5392d6678aa2b68a1796c8113e1aa6
Successfully built elegantrl pybullet
     |████████████████████████████████| 6.3 MB 12.3 MB/s 
  Created wheel for yfina

# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [4]:
from elegantrl.run import *
from elegantrl.agent import *
from elegantrl.envs.FinRL.StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf
from datetime import datetime, timedelta

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [14]:
# Agent
args = Arguments(if_on_policy=True)
args.agent = AgentPPO() # AgentSAC(), AgentTD3(), AgentDDPG()
args.agent.if_use_gae = True
args.agent.lambda_entropy = 0.04

# Environment
tickers = [
  'AAPL', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ALGN', 'AMAT', 'AMD', 'AMGN',
  'AMZN', 'ASML', 'ATVI', 'BIIB', 'BKNG', 'BMRN', 'CDNS', 'CERN', 'CHKP', 'CMCSA',
  'COST', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'DLTR', 'EA', 'EBAY', 'FAST',
  'FISV', 'GILD', 'HAS', 'HSIC', 'IDXX', 'ILMN', 'INCY', 'INTC', 'INTU', 'ISRG',
  'JBHT', 'KLAC', 'LRCX', 'MAR', 'MCHP', 'MDLZ', 'MNST', 'MSFT', 'MU', 'MXIM',
  'NLOK', 'NTAP', 'NTES', 'NVDA', 'ORLY', 'PAYX', 'PCAR', 'PEP', 'QCOM', 'REGN',
  'ROST', 'SBUX', 'SIRI', 'SNPS', 'SWKS', 'TTWO', 'TXN', 'VRSN', 'VRTX', 'WBA',
  'WDC', 'WLTW', 'XEL', 'XLNX']  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1e2
initial_capital = 1e6
initial_stocks = np.zeros(len(tickers), dtype=np.float32)
buy_cost_pct = 0.
sell_cost_pct = 0.
start_date = '2008-01-01'
start_eval_date = (datetime.now() - timedelta(days=365*2)).strftime("%F")
end_eval_date = datetime.now().strftime('%F')

args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False)
args.env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True)

args.env.target_reward = 3
args.env_eval.target_reward = 3

# Hyperparameters
args.gamma = gamma
args.break_step = int(5e5)
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = args.max_step * 4
args.batch_size = 2 ** 9
args.repeat_times = 2 ** 3
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = False
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)

| FeatureEngineer: start processing data (2 minutes)
| YahooDownloader: start downloading data (1 minute)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 complet

In [15]:
train_and_evaluate(args) # the training process will terminate once it reaches the target reward.

| GPU id: 0, cwd: ./AgentPPO/StockTradingEnv-v1_0
| Remove history
ID      Step      MaxR |    avgR      stdR       objA      objC |  avgS  stdS
0   5.82e+03      1.66 |
0   5.82e+03      1.66 |    1.66      0.01       0.22     11.18 |   501     0
0   1.16e+04      1.77 |
0   1.16e+04      1.77 |    1.77      0.01       0.21      7.80 |   501     0
0   1.75e+04      1.77 |    1.56      0.01       0.21      5.73 |   501     0
0   2.91e+04      1.77 |    1.56      0.03       0.19      5.45 |   501     0
0   4.08e+04      1.77 |    1.62      0.02       0.21      5.05 |   501     0
0   5.24e+04      1.77 |    1.75      0.01       0.22      2.84 |   501     0
0   6.40e+04      1.82 |
0   6.40e+04      1.82 |    1.82      0.01       0.24      3.26 |   501     0
0   6.99e+04      1.92 |
0   6.99e+04      1.92 |    1.92      0.01       0.24      2.66 |   501     0
0   7.57e+04      1.92 |
0   7.57e+04      1.92 |    1.92      0.01       0.16      2.91 |   501     0
0   8.15e+04      1.92 |    

In [17]:
args = Arguments(if_on_policy=True)
args.agent = AgentPPO()
args.env = StockTradingEnv(cwd='./', if_eval=True)
args.if_remove = False
args.cwd = './AgentPPO/StockTradingEnv-v1_0'
args.init_before_training()

args.env.draw_cumulative_return(args, torch)

| GPU id: 0, cwd: ./AgentPPO/StockTradingEnv-v1_0
Loaded act: ./AgentPPO/StockTradingEnv-v1_0


[0.9990154585538652,
 0.9939676565366512,
 0.9892317974246928,
 0.9809177066202849,
 0.9539561289623238,
 0.9677042073988965,
 0.9696054008135447,
 0.9926433963632146,
 0.9826899113497164,
 0.9707976543071016,
 0.9906301253921965,
 0.9572488822286886,
 0.9604170299459325,
 0.9721401151355177,
 0.9831418981019812,
 0.9751530813731024,
 0.9850430724601272,
 0.9819414951674409,
 0.9566916982494287,
 0.9686704682435456,
 0.9719381000460583,
 0.9759036499276201,
 0.9883400342362441,
 0.9899605802067813,
 0.9839025370877098,
 0.9951246552121262,
 1.0146140689213838,
 1.0147230883451073,
 1.0057485475138535,
 1.001922145586652,
 1.00471337198954,
 1.0068449649947526,
 1.0072388441209612,
 0.9981255743694654,
 1.0039269357432812,
 1.002491313779388,
 1.0036386654693332,
 0.9983582345079544,
 0.9976714370489577,
 0.9840163745439858,
 0.9937896078933607,
 0.9888683616611782,
 0.9775037999003917,
 0.9869442684582314,
 0.9801573251846079,
 0.9681246813382149,
 0.9775388321555651,
 0.98819275907549

Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [18]:
!rm -rf /content/drive/MyDrive/trading/AgentPPO/ 
!cp -R ./AgentPPO /content/drive/MyDrive/trading/